### Import Dependencies

In [1]:
import random
random.seed(0)

In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.transforms import v2

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Pawel\anaconda3\envs\env_torch\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Constant Variables

In [3]:
DATASET_PATH = './knee-osteoarthritis'

In [4]:
TRAIN_PATH = f'{DATASET_PATH}/train'
VAL_PATH = f'{DATASET_PATH}/val'
TEST_PATH = f'{DATASET_PATH}/test'
AUTO_TEST_PATH = f'{DATASET_PATH}/auto_test'

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cpu


### Dataset 

In [6]:
from src.dataset.augmented_dataset import get_KneeOsteoarthritis_Edges

train_dataset = get_KneeOsteoarthritis_Edges(TRAIN_PATH)
val_dataset = get_KneeOsteoarthritis_Edges(VAL_PATH)

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [7]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

In [8]:
row = train_dataset[1]
normal_ex = row[0]
augmented_ex = row[1]
print(normal_ex.shape, augmented_ex.shape)

torch.Size([3, 256, 256]) torch.Size([1, 224, 224])


### Data Loader

In [9]:
from torch.utils.data import DataLoader
BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

### Model

In [10]:
# class CustomModel(nn.Module):
#     def __init__(self, num_classes: int = 5, dropout: float = 0.5) -> None:
#         super().__init__()
        
#         BASE_SIZE = 32
        
#         # Images
#         self.images_features = nn.Sequential(
#             nn.Conv2d(3, BASE_SIZE, kernel_size=11, stride=4, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(BASE_SIZE, BASE_SIZE*2, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(BASE_SIZE*2, BASE_SIZE*4, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(BASE_SIZE*4, BASE_SIZE*3, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(BASE_SIZE*3, BASE_SIZE*2, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#         )
#         self.images_avgpool = nn.Sequential(
#           # nn.AdaptiveAvgPool2d((6, 6)),
#           nn.Flatten()
#         )
#         self.images_classifier = nn.Sequential(
#             nn.Dropout(p=dropout),
#             nn.Linear(BASE_SIZE*2 * 7 * 7, BASE_SIZE*3),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=dropout),
#             nn.Linear(BASE_SIZE*3, BASE_SIZE),
#             nn.ReLU(inplace=True),
#             nn.Linear(BASE_SIZE, num_classes),
#         )
        
#         EDGES_SIZE = 32
#         # Edges
#         self.edges_features = nn.Sequential(
#             nn.Conv2d(3, EDGES_SIZE, kernel_size=11, stride=4, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(EDGES_SIZE, EDGES_SIZE*2, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(EDGES_SIZE*2, EDGES_SIZE*4, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(EDGES_SIZE*4, EDGES_SIZE*3, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(EDGES_SIZE*3, EDGES_SIZE*2, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#         )
#         self.edges_avgpool = (
#           # nn.AdaptiveAvgPool2d((6, 6)),
#           nn.Flatten()
#         )
#         self.edges_classifier = nn.Sequential(
#             nn.Dropout(p=dropout),
#             nn.Linear(EDGES_SIZE*2 * 7 * 7, EDGES_SIZE*3),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=dropout),
#             nn.Linear(EDGES_SIZE*3, EDGES_SIZE),
#             nn.ReLU(inplace=True),
#             nn.Linear(EDGES_SIZE, num_classes),
#         )
        
#         self.outputCombiner = nn.Sequential(
#             nn.Linear(2 * num_classes, num_classes),
#         )

#     def forward(self, image: torch.Tensor, edges: torch.Tensor) -> torch.Tensor:
        
#         images = self.images_features(image)
#         # print(images.shape)
#         images = self.images_avgpool(images)
#         # print(images.shape)
#         images = self.images_classifier(images)
        
#         # Edges
#         edges = self.edges_features(image)
#         # print(edges.shape)
#         edges = self.edges_avgpool(edges)
#         # print(edges.shape)
#         edges = self.edges_classifier(edges)
        
#         # Combining outputs
#         concated = torch.cat((images, edges), 1)
#         res = self.outputCombiner(concated)
        
#         return res

In [61]:
class CustomModel(nn.Module):
    def __init__(self, num_classes: int = 5, dropout: float = 0.5) -> None:
        super().__init__()
        
        self.edgesClassifier = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.Dropout(p=dropout*0.4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=dropout*0.6),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout*0.8),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(32 * 6 * 6, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(64, num_classes),
        )

    def forward(self, image: torch.Tensor, edges: torch.Tensor) -> torch.Tensor:
        
        # Edges
        edges = self.edgesClassifier(edges)
        
        return edges

In [62]:
# from src.models.custom import CustomModel

model = CustomModel(3, 0.5)
model = model.to(device)

In [63]:
# print(sum(p.numel() for p in net.classifier.parameters()) ,sum(p.numel() for p in net.edgesClassifier.parameters()) )
print(sum(p.numel() for p in model.parameters()))

trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())
print(sum(p.numel() for p in trainable_parameters))

202723
202723


### Training Loop

#### Setting optimizer

In [64]:
import torch.optim as optim
from src.other import getClassesFrequency
# criterion = nn.NLLLoss()

class_weights = getClassesFrequency(train_dataset)
weights_tensor = torch.Tensor(list(class_weights.values())).to(device)

criterion = nn.CrossEntropyLoss(weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [65]:
def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']

#### Setting Logger

In [66]:
EXP_NAME = "images_downscaled_alexnet"

In [67]:
from torch.utils.tensorboard import SummaryWriter

logger = SummaryWriter(log_dir=f"logs/experiments/{EXP_NAME}")

In [68]:
epochCounter = 0

#### Training Loop

In [69]:
from src.validation import validate

def train_many(model, epochs_nr, regularization_type = "L2", lambda_reg=0.01):
    global epochCounter
    
    for epoch in range(0, epochs_nr):  # loop over the dataset multiple times
        epoch_correct = 0
        epoch_samples = 0
        epoch_batches = 0
        running_loss = 0.0
    
        for i, data in enumerate(train_loader, 0):

            # get the inputs; data is a list of [inputs, labels]
            images, edges, labels = data
            images = images.to(device)
            edges = edges.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(images, edges)
            loss = criterion(outputs, labels)
            
            # Apply L1 regularization
            if regularization_type == 'L1':
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss += lambda_reg * l1_norm
                
            # Apply L2 regularization
            elif regularization_type == 'L2':
                l2_norm = sum(p.pow(2).sum() for p in model.parameters())
                loss += lambda_reg * l2_norm
                
            loss.backward()
            
            optimizer.step()
            
            # Changing outputs (logits) to labels
            outputs_clear = outputs.max(1).indices
            
            epoch_correct += (outputs_clear == labels).float().sum()
            epoch_samples += len(outputs)
            epoch_batches +=1
            
            running_loss += loss.item()
        
        tAccuracy = epoch_correct / epoch_samples * 100
        tLoss = running_loss / epoch_batches
        
        # Validation
        vAccuracy, vLoss = validate(model, val_loader, criterion, device)
        
        logger.add_text("REGULARIZATION_TYPE", regularization_type, global_step=epochCounter)
        logger.add_scalar("REGULARIZATION_LAMBDA", lambda_reg, global_step=epochCounter)
        logger.add_scalar("learning_rate", get_lr(optimizer), global_step=epochCounter)
        
        logger.add_scalar("Accuracy/train", tAccuracy, global_step=epochCounter)
        logger.add_scalar("Loss/train", tLoss, global_step=epochCounter)
        logger.add_scalar("Accuracy/validation", vAccuracy, global_step=epochCounter)
        logger.add_scalar("Loss/validation", vLoss, global_step=epochCounter)
        
        print(f'Epoch {epochCounter}: Training: accuracy: {tAccuracy:.3f}%, loss: {tLoss:.3f}; Validation: accuracy: {vAccuracy:.3f}%, loss: {vLoss:.3f}')
        
        epochCounter += 1
        
    print('Finished Training')

### Training Model

In [70]:
train_many(model, 50)

KeyboardInterrupt: 